In [59]:
import pandas as pd

In [60]:
# Load the files into pandas dataframes
file_1_path = "Data/Revised/Geography/Lookup/11-11-MSOA.csv"
file_2_path = "Data/Revised/Population/MSOA_2011.csv"
file_3_path = "Data/Revised/Population/LSOA_2011.csv"

# Read the CSV files
file_1 = pd.read_csv(file_1_path)
file_2 = pd.read_csv(file_2_path)
file_3 = pd.read_csv(file_3_path)

# Rename columns for merging
file_2_renamed = file_2.rename(columns={"Persons_All usual residents": "MSOA"})
file_3_renamed = file_3.rename(columns={"Persons_All usual residents": "LSOA"})

# Merge with '11-11-MSOA' DataFrame on 'MSOA11CD' and 'MSOA Code'
merged_data = file_1.merge(file_2_renamed[['MSOA Code', 'MSOA']], left_on='MSOA11CD', right_on='MSOA Code', how='left')

# Merge with '11-11-MSOA' DataFrame on 'LSOA11CD' and 'LSOA Code'
merged_data = merged_data.merge(file_3_renamed[['LSOA Code', 'LSOA']], left_on='LSOA11CD', right_on='LSOA Code', how='left')

# Drop unnecessary columns after merge
merged_data = merged_data.drop(columns=['MSOA Code', 'LSOA Code'])

# Reorder the columns to place 'MSOA' as the last column
merged_data = merged_data[['LSOA11CD', 'LSOA11NM', 'MSOA11CD', 'MSOA11NM', 'LSOA', 'MSOA']]

# Convert 'LSOA' and 'MSOA' columns to numeric by removing commas and converting to integers
merged_data['LSOA'] = merged_data['LSOA'].str.replace(',', '').astype(int)
merged_data['MSOA'] = merged_data['MSOA'].str.replace(',', '').astype(int)

# Calculate the proportion of LSOA over MSOA and round to two decimal places
merged_data['Proportion'] = (merged_data['LSOA'] / merged_data['MSOA']).round(2)

In [61]:
# Load the IMD data
imd_file_path = "Data/Revised/IMD/IMD.csv"
imd_data = pd.read_csv(imd_file_path)

# Merging the existing merged data with the IMD data on 'LSOA11CD' and 'LSOA_Code_2011'
final_merged_data = merged_data.merge(imd_data[['LSOA_Code_2011', 'Index of Multiple Deprivation (IMD) Score']], 
                                      left_on='LSOA11CD', 
                                      right_on='LSOA_Code_2011', 
                                      how='left')

# Dropping the redundant 'LSOA_Code_2011' column after merge
final_merged_data.drop(columns=['LSOA_Code_2011'], inplace=True)

# Calculating the product of 'Proportion' and 'IMD_Decile' to create a new column 'D'
final_merged_data['S'] = (final_merged_data['Proportion'] * final_merged_data['Index of Multiple Deprivation (IMD) Score']).round(2)

# Display the updated DataFrame with the new 'Decile' column
print(final_merged_data.head())

    LSOA11CD             LSOA11NM   MSOA11CD            MSOA11NM  LSOA  MSOA  \
0  E01000001  City of London 001A  E02000001  City of London 001  1465  7375   
1  E01000003  City of London 001C  E02000001  City of London 001  1346  7375   
2  E01000002  City of London 001B  E02000001  City of London 001  1436  7375   
3  E01032739  City of London 001F  E02000001  City of London 001  1044  7375   
4  E01032740  City of London 001G  E02000001  City of London 001  1099  7375   

   Proportion  Index of Multiple Deprivation (IMD) Score     S  
0        0.20                                      6.208  1.24  
1        0.18                                     19.402  3.49  
2        0.19                                      5.143  0.98  
3        0.14                                     13.584  1.90  
4        0.15                                     15.841  2.38  


In [62]:
# Grouping by 'MSOA11NM' and summing the 'D' values for each group
score_sum = final_merged_data.groupby('MSOA11NM')['S'].sum().reset_index()

# Renaming the summed column to 'Decile'
score_sum.rename(columns={'S': 'Score'}, inplace=True)

# Display the updated DataFrame with the new 'Decile' column
print(score_sum.head())

                   MSOA11NM  Score
0  Barking and Dagenham 001  34.72
1  Barking and Dagenham 002  25.10
2  Barking and Dagenham 003  23.43
3  Barking and Dagenham 004  30.40
4  Barking and Dagenham 006  36.18


In [63]:
# Merging the summed 'Decile' values back to the original dataframe, aligning on 'MSOA11NM'
final_data_with_score = final_merged_data.merge(score_sum, on='MSOA11NM', how='left')

# Display the final dataframe
print(final_data_with_score)

       LSOA11CD             LSOA11NM   MSOA11CD            MSOA11NM  LSOA  \
0     E01000001  City of London 001A  E02000001  City of London 001  1465   
1     E01000003  City of London 001C  E02000001  City of London 001  1346   
2     E01000002  City of London 001B  E02000001  City of London 001  1436   
3     E01032739  City of London 001F  E02000001  City of London 001  1044   
4     E01032740  City of London 001G  E02000001  City of London 001  1099   
...         ...                  ...        ...                 ...   ...   
4830  E01004762     Westminster 011E  E02000970     Westminster 011  2070   
4831  E01004763     Westminster 013B  E02000972     Westminster 013  2127   
4832  E01004765     Westminster 013D  E02000972     Westminster 013  2023   
4833  E01004739     Westminster 024E  E02000983     Westminster 024  1348   
4834  E01033598     Westminster 009F  E02000968     Westminster 009  1242   

       MSOA  Proportion  Index of Multiple Deprivation (IMD) Score     S  \

In [64]:
# Selecting only the required columns: 'MSOA11CD', 'MSOA11NM', and 'Decile'
final_selected_data = final_data_with_score[['MSOA11CD', 'MSOA11NM', 'Score']]

# Dropping duplicate rows based on 'MSOA11CD' and 'MSOA11NM'
final_unique_data = final_selected_data.drop_duplicates(subset=['MSOA11CD', 'MSOA11NM'])

# Rounding the 'Decile' values to the nearest integer using .loc to avoid the SettingWithCopyWarning
final_unique_data.loc[:, 'Score'] = final_unique_data['Score'].round(2)

# Display the resulting unique dataframe
print(final_unique_data)

       MSOA11CD                  MSOA11NM  Score
0     E02000001        City of London 001  13.71
6     E02000017  Barking and Dagenham 016  27.52
8     E02000016  Barking and Dagenham 015  33.88
13    E02000010  Barking and Dagenham 009  34.95
17    E02000014  Barking and Dagenham 013  36.56
...         ...                       ...    ...
4798  E02000979           Westminster 020  20.52
4800  E02000977           Westminster 018  20.76
4806  E02000981           Westminster 022  14.43
4808  E02000980           Westminster 021  23.38
4820  E02000969           Westminster 010  38.72

[983 rows x 3 columns]


In [65]:
# Save the filtered DataFrame to a new CSV file
final_unique_data.to_csv('Data/Revised/Migration/IMD_2019.csv', index=False)